### LangChain's Runnable: The Building Blocks of Composable AI
In the LangChain framework, a Runnable is a fundamental abstraction representing any unit of work that can be invoked, composed into chains, and executed. This standardized interface is at the heart of the LangChain Expression Language (LCEL), enabling developers to construct complex AI workflows by connecting various components in a modular and intuitive way.

At its core, a Runnable provides a consistent set of methods for interaction, including invoke for a single input, batch for multiple inputs, and stream for processing data as it becomes available. This uniformity allows for the seamless chaining of different components, from language models and prompt templates to data retrievers and output parsers. The primary mechanism for composing Runnables is the pipe operator (|), which passes the output of one Runnable as the input to the next.

### Task-Specific Runnables: The "Doers" of the Chain
As their name suggests, Task-Specific Runnables are the core components that perform a particular, well-defined task within your AI application. These are the "workers" of your chain, responsible for the primary logic and processing. Think of them as the individual tools in your AI toolbox.

Examples of Task-Specific Runnables include:

- LLMs and Chat Models: Such as ChatOpenAI, these are responsible for generating text-based responses.[3][5]
- PromptTemplates: These format user input into a specific prompt structure that a language model can understand.
- Retrievers: These components fetch relevant documents or data from a vector store or other data source.[5]
- Output Parsers: These structure the output from a language model into a more usable format, like JSON or a Python object.

Essentially, any core LangChain component that has been adapted to the Runnable interface to be seamlessly integrated into a chain falls under this category.[5]

### Runnable Primitives: The "Orchestrators" of the Chain
In contrast to the "doers," Runnable Primitives are the "orchestrators." They are helper tools that don't perform AI tasks themselves but rather control and structure the flow of data and the execution of other Runnables within a chain.[4] They provide the logic for how the Task-Specific Runnables interact with each other.

### RunnableSequence
RunnableSequence in LangChain

A RunnableSequence is a type of Runnable Primitive.

It allows you to chain multiple Runnables together in a pipeline — where the output of one Runnable becomes the input of the next.

It’s the LangChain equivalent of a function composition or pipeline of steps.

👉 Think of it like Unix pipes (|) or scikit-learn Pipelines.

### Key Points about RunnableSequence

Defined using RunnableSequence.from([...])

Takes a list of steps (Runnables)

Executes them in order

Useful when you want to build custom workflows step by step

In [1]:
from langchain_google_genai import GoogleGenerativeAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from dotenv import load_dotenv
from langchain.schema.runnable import RunnableSequence

load_dotenv()

prompt1 = PromptTemplate(
    template='Write a joke about {topic}',
    input_variables=['topic']
)

model = GoogleGenerativeAI(model="gemini-2.5-flash")

parser = StrOutputParser()

prompt2 = PromptTemplate(
    template='Explain the following joke - {text}',
    input_variables=['text']
)

chain = RunnableSequence(prompt1, model, parser, prompt2, model, parser)

print(chain.invoke({'topic':'AI'}))

These jokes cleverly play on common understandings and current limitations of Artificial Intelligence, using anthropomorphism (giving human qualities to AI) to highlight their machine nature.

Let's break down each one:

---

**Option 1 (Focus on training data):**

> Why did the AI go to therapy?
> Because it had too many unresolved issues from its training data.

*   **The Human Connection:** Humans go to therapy to deal with "unresolved issues" – past experiences, traumas, or conflicting beliefs that affect their current behavior and well-being.
*   **The AI Connection:** AI models learn from vast amounts of "training data." If this data is biased, contradictory, incomplete, or contains problematic information, the AI will internalize those "issues" and reflect them in its outputs (e.g., generating biased responses, making factual errors, or struggling with certain types of queries).
*   **The Humor:** The humor comes from the direct analogy. Just as human "issues" stem from their li

### What is RunnableParallel?

- RunnableParallel is another Runnable Primitive in LangChain.

- Unlike RunnableSequence (which runs steps one after the other, serially),
RunnableParallel runs multiple Runnables at the same time (in parallel).

- It returns a dictionary of results where each key corresponds to the Runnable you defined.

In [5]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnableParallel
from dotenv import load_dotenv

load_dotenv()

LLM = ChatGoogleGenerativeAI(temperature=0,model="gemini-2.5-flash")

english_prompt = ChatPromptTemplate.from_template(
    "Translate the following English : {text}")

french_prompt = ChatPromptTemplate.from_template("Translate the following French: {text}")

parallel_chain = RunnableParallel(
    english=english_prompt|LLM,
    french=french_prompt|LLM
)

result = parallel_chain.invoke({"text": "Hello, how are you?"})
print(result)

{'english': AIMessage(content='Here are a few ways to translate "Hello, how are you?" into French, depending on the level of formality and who you are speaking to:\n\n1.  **Formal (to someone you don\'t know well, an elder, or in a professional setting):**\n    *   **Bonjour, comment allez-vous ?**\n        *   *Bonjour* = Hello/Good day\n        *   *Comment allez-vous ?* = How are you? (using "vous" for politeness/formality)\n\n2.  **Informal (to a friend, family member, or someone your age):**\n    *   **Salut, comment vas-tu ?**\n        *   *Salut* = Hi/Hello (more casual)\n        *   *Comment vas-tu ?* = How are you? (using "tu" for familiarity)\n\n3.  **Very Common & Versatile (can be used in many situations, often informal but can be polite depending on tone):**\n    *   **Bonjour, ça va ?** (during the day)\n    *   **Salut, ça va ?** (more informal, anytime)\n        *   *Ça va ?* literally means "It goes?" but is widely used to ask "How are you?" or "Are you okay?". The res

### What is RunnablePassthrough?

- It’s the simplest Runnable Primitive in LangChain.

- As the name suggests, it passes the input straight through to the output without changing it.

- Useful when:

- You need a placeholder in a chain

- You want to combine inputs with other runnables (merge original input with computed values)

- You’re debugging a pipeline and just want to see the raw input.

In [6]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from dotenv import load_dotenv
from langchain.schema.runnable import RunnableSequence, RunnableParallel, RunnablePassthrough

load_dotenv()

prompt1 = PromptTemplate(
    template='Write a joke about {topic}',
    input_variables=['topic']
)

model = ChatGoogleGenerativeAI(temperature=0,model="gemini-2.5-flash")

parser = StrOutputParser()

prompt2 = PromptTemplate(
    template='Explain the following joke - {text}',
    input_variables=['text']
)

joke_gen_chain = RunnableSequence(prompt1, model, parser)

parallel_chain = RunnableParallel({
    'joke': RunnablePassthrough(),
    'explanation': RunnableSequence(prompt2, model, parser)
})

final_chain = RunnableSequence(joke_gen_chain, parallel_chain)

print(final_chain.invoke({'topic':'cricket'}))

{'joke': "Why did the cricket (the insect) get kicked out of the cricket match?\n\nBecause he kept **chirping** about the umpire's decisions!", 'explanation': 'This joke works because of a clever play on words, specifically the word "chirping":\n\n1.  **"Cricket" (the insect) vs. "Cricket" (the sport):** The setup immediately creates a humorous image by putting an insect into a human sporting event.\n\n2.  **"Chirping" (literal meaning):** A cricket (the insect) is famous for the high-pitched, repetitive sound it makes, which is called "chirping."\n\n3.  **"Chirping" (figurative meaning):** When applied to humans, "chirping" (or "to chirp about something") means to complain, grumble, or make persistent, often annoying, critical remarks. If someone is "chirping about the umpire\'s decisions," it means they are constantly complaining and criticizing the umpire\'s calls.\n\n**The Humor:**\n\nThe joke combines these meanings:\n\n*   It takes the literal sound an insect cricket makes ("chir

### What is RunnableLambda?

- A Runnable Primitive in LangChain.

- It lets you wrap a Python function (lambda or normal function) into a Runnable.

- This makes it possible to use custom logic inside an LCEL pipeline.

👉 Think of it as:
RunnableLambda = bridge between your own Python code ↔ LangChain Runnable system

In [7]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from dotenv import load_dotenv
from langchain.schema.runnable import RunnableSequence, RunnableLambda, RunnablePassthrough, RunnableParallel

load_dotenv()

def word_count(text):
    return len(text.split())

prompt = PromptTemplate(
    template='Write a joke about {topic}',
    input_variables=['topic']
)

model = ChatGoogleGenerativeAI(temperature=0,model="gemini-2.5-flash")

parser = StrOutputParser()

joke_gen_chain = RunnableSequence(prompt, model, parser)

parallel_chain = RunnableParallel({
    'joke': RunnablePassthrough(),
    'word_count': RunnableLambda(word_count)
})

final_chain = RunnableSequence(joke_gen_chain, parallel_chain)

result = final_chain.invoke({'topic':'AI'})

final_result = """{} \n word count - {}""".format(result['joke'], result['word_count'])

print(final_result)

Here are a few options for a joke about AI:

**Option 1 (Literal Interpretation):**
Why did the AI break up with its girlfriend?
Because she had too many "unstructured data points."

**Option 2 (Hallucinations):**
My AI is great, but it keeps making up facts. I asked it for the capital of France, and it said "Bologna, but only if you're wearing socks."

**Option 3 (Optimization):**
Why did the AI get fired from the bakery?
It kept trying to optimize the recipe for maximum flour efficiency, and the cakes tasted like cardboard.

**Option 4 (Classic setup):**
Why did the AI cross the road?
To optimize its pathfinding algorithm. 
 word count - 107


### What is RunnableBranch?

RunnableBranch is a control-flow primitive in LangChain.

It works like an if-elif-else statement inside a Runnable pipeline.

In [8]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from dotenv import load_dotenv
from langchain.schema.runnable import RunnableSequence, RunnableParallel, RunnablePassthrough, RunnableBranch, RunnableLambda

load_dotenv()

prompt1 = PromptTemplate(
    template='Write a detailed report on {topic}',
    input_variables=['topic']
)

prompt2 = PromptTemplate(
    template='Summarize the following text \n {text}',
    input_variables=['text']
)

model = ChatGoogleGenerativeAI(temperature=0,model="gemini-2.5-flash")

parser = StrOutputParser()

report_gen_chain = prompt1 | model | parser

branch_chain = RunnableBranch(
    (lambda x: len(x.split())>300, prompt2 | model | parser),
    RunnablePassthrough()
)

final_chain = RunnableSequence(report_gen_chain, branch_chain)

print(final_chain.invoke({'topic':'Russia vs Ukraine'}))


The Russia-Ukraine conflict, which escalated into a full-scale invasion on February 24, 2022, is the largest conventional war in Europe since World War II. Rooted in centuries of complex historical ties, post-Soviet geopolitical shifts, and Ukraine's aspirations for closer integration with the West, the conflict saw Russia annex Crimea in 2014 and support separatists in Donbas, leading to an eight-year low-intensity war before the 2022 invasion.

Russia's initial attempt for a swift victory and regime change in Kyiv failed due to fierce Ukrainian resistance. The conflict evolved into a protracted war of attrition, with Russia shifting focus to Donbas. Ukraine launched successful counter-offensives in late 2022 (Kharkiv, Kherson), but a 2023 offensive has seen slow progress against fortified Russian lines, leading to a current military stalemate.

Key players include Ukrainian President Volodymyr Zelenskyy, Russian President Vladimir Putin, and international actors like the United State